In [26]:
from tensorflow import keras
import tensorflow as tf
from matplotlib import pyplot as plt
from keras.applications import inception_v3
from keras import Model
import numpy as np

I load an InceptionV3 model with imagenet weights.

In [27]:
model = inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=())

In [28]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_282 (Conv2D)            (None, None, None,   864         ['input_4[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization_282 (Batch  (None, None, None,   96         ['conv2d_282[0][0]']             
 Normalization)                 32)                                                    

Identify layers for which you want to maximize activations. Each layer will have a weighted contribution to the loss. The weights are also defined.

In [33]:
layer_settings = {
    'mixed4': 1.0,
    'mixed5': 1.5,
    'mixed6': 2.0,
    'mixed7': 2.5
}

outputs_dict = dict(
    [
        (layer.name, layer.output) for layer in [
            model.get_layer(name) for name in layer_settings.keys()
        ]
    ]
)

Define a model to return the activations of the layers above.

In [34]:
feature_extractor = Model(model.inputs, outputs_dict)

Define the loss. For each layer, the loss contains a term which is the mean of the L2 norm of the activations from the layer. Note this is similar to a common example of gradient ascent in which only the output of one filter is maximized.

In [35]:
def compute_loss(img):
    features = feature_extractor(img)
    loss = tf.zeros(())
    for name in features.keys():
        coeff = layer_settings[name]
        activation = features[name]
        loss += coeff * tf.reduce_mean(tf.square(activation[:, 2:-2, 2:-2, :])) # Apparently, removing border pixels avoids artifacts
    return loss

Define the gradient ascent process for an octave. Note this is still similar to the simpler gradient ascent case.

In [44]:
@tf.function
def gradient_ascent_step(img, learning_rate):
    '''
    A single update in the gradient ascent process
    '''
    with tf.GradientTape() as tape:
        tape.watch(img)
        loss = compute_loss(img)
    grads = tape.gradient(loss, img)
    grads = tf.math.l2_normalize(grads) # This is a trick to help the optimization process
    img += learning_rate * grads
    return loss, img

def gradient_ascent_loop(img, iterations, learning_rate, max_loss=None):
    '''
    A gradient ascent loop for a single octave
    '''
    for i in range(iterations):
        loss, img = gradient_ascent_step(img, learning_rate)
        if max_loss is not None and loss > max_loss: # prevents image artifacts
            break
    return img

Define the upscaling process. DeepDream performs the gradient ascent process at a number of different scales. These scalews are called "octaves." It performs gradient ascent first at the smallest scale, then it upscales the image and performs gradient ascent again, etc.

In [65]:
step = 20
num_octaves = 3
octave_scale = 1.1
iterations = 30
max_loss = 15

First, some utility functions.

In [66]:
def preprocess_image(img_path):
    img = keras.utils.load_img(img_path, target_size=(300,300))
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(img):
    img = img.reshape((img.shape[1], img.shape[2], 3))
    # undo inception preprocessing
    img /= 2.0
    img += 0.5
    img *= 255
    img = np.clip(img, 0, 255).astype(int)
    return img

In [69]:
img_path = 'eagle.jpg'
img_name = img_path.split('.')[0]
original_img =  preprocess_image(img_path)
original_shape = original_img.shape[1:3]

# Create the shapes for each octave
successive_shapes = [original_shape]
for i in range(1, num_octaves):
    shape = tuple(int(dim/(octave_scale**i)) for dim in original_shape)
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]

shrunk_original_img = tf.image.resize(original_img, successive_shapes[0])

img = tf.identity(original_img)

# Perform gradient ascent for each octave
for i, shape in enumerate(successive_shapes):
    print(f'Performing octave {i}')
    img = tf.image.resize(img, shape)
    img = gradient_ascent_loop(img, iterations, learning_rate=step, max_loss=max_loss)

    # upscaling
    upscaled_shrunk_image = tf.image.resize(shrunk_original_img, shape)
    downscaled_original = tf.image.resize(original_img, shape)

    lost_detail = downscaled_original - upscaled_shrunk_image

    img += lost_detail

    shrunk_original_img = tf.image.resize(original_img, shape)

keras.utils.save_img(img_name+'_dream.jpg', deprocess_image(img.numpy()))

Performing octave 0
Performing octave 1
Performing octave 2
